In [0]:
spark.sql("USE globalretail_silver")
spark.sql("""
    CREATE TABLE IF NOT EXISTS silver_customers (
    customer_id STRING,
    name STRING,
    email STRING,
    country STRING,
    customer_type STRING,
    registration_date DATE,
    age INT,
    gender STRING,
    total_purchases INT,
    customer_segment STRING,
    days_since_registration INT,
    last_updated TIMESTAMP)
""")

In [0]:
last_processed_df = spark.sql("SELECT MAX(last_updated) as last_processed FROM silver_customers")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW bronze_incremental AS
SELECT *
FROM globalretail_bronze.bronze_customer c where  c.ingestion_timestamp > '{last_processed_timestamp}'
""")

In [0]:
spark.sql("select * from bronze_incremental").show()

Validate email addresses (null or not null)

Valid age between 18 to 100

Create customer_segment as total_purchases > 10000 THEN 'High Value' if > 5000 THEN 'Medium Value'  ELSE 'Low Value'

Days since user is registered in the system

Remove any junk records where total_purchase is negative number

In [0]:
"""
import dlt
from pyspark.sql.functions import col

@dlt.table
@dlt.expect("valid_email", "email LIKE '%@%.%'")
@dlt.expect("non_null_age", "age IS NOT NULL")
def silver_customers():
    return spark.read.table("globalretail_bronze.bronze_customer")
"""

In [0]:
# Transformation
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental AS
SELECT
customer_id,
name,
email,
country,
customer_type,
registration_date,
age,
gender,
total_purchases,
CASE
WHEN total_purchases > 10000 THEN 'High Value'
WHEN total_purchases > 5000 THEN 'Medium Value'
ELSE 'Low Value'
END AS customer_segment,
DATEDIFF (CURRENT_DATE(), registration_date) AS days_since_registration,
CURRENT_TIMESTAMP() as last_updated
FROM bronze_incremental
WHERE
age BETWEEN 18 AND 100
AND email IS NOT NULL
AND total_purchases >= 0
""")

In [0]:
display(spark.sql("select * from silver_incremental"))

In [0]:
spark.sql("""
MERGE INTO silver_customers target
USING silver_incremental source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
UPDATE SET *
WHEN NOT MATCHED THEN
INSERT *
""")

In [0]:
spark.sql("select * from silver_customers").show()

In [0]:
spark.sql("select count(*) from silver_customers").show()